In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
print(tf.version)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
%cd keras-retinanet/

!pip install .

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
ANNOTATIONS_FILE = '' # Enter your annotations csv file
CLASSES_FILE = '' #Enter your classes csv file
DATASET_DIR = ''
JPEGImages = ''

In [ ]:
import os
import shutil
import zipfile
import urllib
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import tensorflow.compat.v1 as tf
import tensorflow.keras.backend as K

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
#keras.backend.tensorflow_backend.set_session(get_session())
#K.set_session(get_session())
tf.compat.v1.keras.backend.set_session(get_session)

LOAD TEACHER MODEL

In [ ]:
snap_shots_path = "" # Enter teacher weights snapshots path 
model_path = os.path.join(snap_shots_path, sorted(os.listdir(snap_shots_path), reverse=True)[0])
print(model_path)

# load teacher model
model1 = models.load_model(model_path, backbone_name='resnet50')
model1 = models.convert_model(model1)

# load label to names mapping for visualization purposes
labels_to_names = pandas.read_csv(CLASSES_FILE,header=None).T.loc[0].to_dict()

IMAGE AUGMENTATION

In [ ]:
################################### CUT-OUT #######################################3

import random
import cv2
save_path = '' # Enter save path
path1 = '' # Enter unlabeled dataset path
image_liste1 = os.listdir(path1)
for j in image_liste1:
  test_img_path1 = path1 + j
  img = load_img(test_img_path1)
  image = img_to_array(img)

  cutout_img = image.copy()

  image = preprocess_image(image)
  image, scale = resize_image(image)
  boxes, scores, labels = model1.predict_on_batch(np.expand_dims(image, axis=0))
  boxes /= scale

  cutout_img = cv2.cvtColor(cutout_img, cv2.COLOR_BGR2RGB)

  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
      break    
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])
    mask_w = int((x2 - x1)*0.5)
    mask_h = int((y2 - y1)*0.5)
    mask_x1 = random.randint(x1, x2 - mask_w)
    mask_y1 = random.randint(y1, y2 - mask_h)
    mask_x2 = mask_x1 + mask_w
    mask_y2 = mask_y1 + mask_h
    cv2.rectangle(cutout_img, (mask_x1, mask_y1), (mask_x2, mask_y2), (0, 0, 0), thickness=-1)

    Red = cutout_img[:,:,2]
    Green = cutout_img[:,:,1]
    Blue = cutout_img[:,:,0]

    strong_aug = np.ones((1080,1920,3), dtype=np.uint8)
    a=0.3
    b=1.7
    strong_aug[:,:,0] = Blue*random.uniform(a, b)
    strong_aug[:,:,1] = Green*random.uniform(a, b)
    strong_aug[:,:,2] = Red*random.uniform(a, b) 
  rsz_img = cv2.resize(strong_aug, (1920,1080))
  cv2.imwrite(save_path+"/{}".format(j), rsz_img)
  print(j)

In [ ]:
######################## ADD NOİSE ###############################

def augmentation(test_img_path):
  save_path = ""
  
  import cv2
  from numpy import expand_dims
  from matplotlib import pyplot as plt
  from keras.preprocessing.image import ImageDataGenerator
  from keras.preprocessing.image import load_img
  from keras.preprocessing.image import img_to_array
  import random
  
  #blur = cv2.GaussianBlur(final_result, (7, 7), 0)

  real_img = load_img(test_img_path)
  copy_img = real_img.copy()

  #img = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2HSV )

  x = img_to_array(copy_img)
  samples = expand_dims(x, 0)
  def add_noise(img):
    VARIABILITY = 25
    #deviation = VARIABILITY*random.random()
    deviation = VARIABILITY*0.5
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img
  datagen = ImageDataGenerator(preprocessing_function=add_noise)
                               #horizontal_flip=True
                               #rotation_range=90,
                               #fill_mode='nearest')
  it = datagen.flow(samples, batch_size=1,
                    save_to_dir= save_path, 
                            save_format='jpg')
  batch = it.next()



In [ ]:
pathh = '/content/drive/My Drive/RetinaNet/Dataset/test/'
image_liste = os.listdir(pathh)
for j in image_liste:
  test_img_path = '/content/drive/My Drive/RetinaNet/Dataset/test/' + j
  augmentation(test_img_path)
  if j.endswith('jpg'):
    #img_inference_4_orj(test_img_path)
    print(j)